#### ! REPRODUCIBILITY NOTE: execution of this notebook can be skipped, files created by this notebook are already in  `../metadata` 

In [2]:
from ripe.atlas.cousteau import Probe

In [3]:
import sqlite3
import pandas as pd

In [4]:
with sqlite3.connect('../data/main.db') as conn:
    probes = pd.read_sql('select * from probes', con=conn)

In [5]:
probes.head()

,id,probe_id,country_code
0,1,10003,NL
1,2,10006,GB
2,3,10007,US
3,4,10019,FR
4,5,10023,DE


In [6]:
len(probes)

5229

In [14]:
home_probes = []
coordinates = dict()

for p in probes['probe_id'].unique():
    prb = Probe(id=p)
    coordinates[p] = prb.geometry['coordinates']

    for tag in prb.tags:
        if tag['slug'] == 'home':
            home_probes.append(p)

In [17]:
coordinates

{10003: [4.9375, 52.3685],
 10006: [0.1575, 52.2205],
 10007: [-122.3325, 38.3305],
 10019: [5.2875, 43.5315],
 10023: [13.3785, 52.4985],
 10031: [13.4085, 52.5175],
 10034: [21.0185, 52.1995],
 10047: [14.4795, 50.0305],
 10048: [-1.4405, 50.9415],
 10051: [8.7395, 50.1175],
 10052: [11.5895, 48.1885],
 10058: [5.4395, 43.3415],
 10068: [15.2305, 44.1215],
 10070: [30.5205, 50.4175],
 10080: [139.4315, 35.9175],
 10087: [20.6315, 50.8805],
 10089: [-0.0885, 51.5415],
 10092: [23.4015, 42.6885],
 10095: [174.7115, -36.7705],
 10098: [-0.3785, 51.6015],
 10099: [-84.2785, 34.0215],
 10101: [4.6075, 52.3475],
 10113: [21.4275, 41.9985],
 10131: [20.4685, 44.8075],
 10134: [-0.1915, 51.5895],
 10141: [7.5695, 46.8675],
 10146: [-71.1005, 42.2405],
 10152: [20.4195, 42.0685],
 10176: [19.9105, 42.1705],
 10179: [17.8505, 59.4115],
 10180: [5.3815, 52.1975],
 10185: [-117.1985, 32.8595],
 10188: [6.0815, 50.7815],
 10194: [-87.9985, 41.8795],
 10203: [1.2275, 6.1685],
 10204: [-71.1525, 42

In [25]:
coordinates_df = pd.DataFrame(coordinates).T.reset_index().rename(columns={0 : 'long',
                                                                           1 : 'lat',
                                                                          'index' : 'probe_id'}
                                                                 )

In [26]:
coordinates_df

,probe_id,long,lat
0,10003,4.9375,52.3685
1,10006,0.1575,52.2205
2,10007,-122.3325,38.3305
3,10019,5.2875,43.5315
4,10023,13.3785,52.4985
...,...,...,...
5224,35733,49.5785,37.2785
5225,35734,36.2385,30.5895
5226,35735,-58.6715,-62.2405
5227,35742,31.0095,-29.8015


In [27]:
coordinates_df.to_csv('../metadata/probe-coordinates.csv', index=False, header=True)

In [22]:
home_probes

[10006,
 10092,
 10099,
 10101,
 10134,
 10141,
 10194,
 10214,
 10228,
 10231,
 10250,
 10258,
 10266,
 10301,
 10312,
 10317,
 10322,
 10331,
 10338,
 10386,
 10397,
 10400,
 10441,
 10443,
 10462,
 10473,
 10479,
 10486,
 10497,
 10540,
 10542,
 10553,
 10555,
 10562,
 10568,
 10576,
 10583,
 10584,
 10605,
 10611,
 10645,
 10647,
 10650,
 10654,
 10661,
 10717,
 10729,
 10738,
 10743,
 10750,
 10767,
 10770,
 10774,
 10790,
 10819,
 10843,
 10858,
 10864,
 10866,
 10869,
 10884,
 10890,
 10923,
 10928,
 10933,
 10940,
 10946,
 10967,
 10982,
 11022,
 11054,
 11056,
 11075,
 11090,
 11108,
 11116,
 11145,
 11158,
 11162,
 11175,
 11179,
 11200,
 11209,
 11210,
 11260,
 11266,
 11275,
 11289,
 11323,
 11374,
 11437,
 11499,
 11500,
 11506,
 11523,
 11526,
 11529,
 11576,
 11577,
 11583,
 11592,
 11608,
 11610,
 11629,
 11642,
 11673,
 11713,
 11722,
 11758,
 11760,
 11761,
 11774,
 11794,
 11828,
 11849,
 11863,
 11873,
 11883,
 11902,
 11914,
 11950,
 11967,
 11968,
 11990,
 11992,


In [23]:
len(home_probes)

1439

In [24]:
home_probes_ids = pd.Series(home_probes)

In [25]:
home_probes_ids

0       10006
1       10092
2       10099
3       10101
4       10134
        ...  
1434    35689
1435    35695
1436    35699
1437    35701
1438    35724
Length: 1439, dtype: int64

In [29]:
home_probes_ids.to_csv('../metadata/home-probes-ids.csv', index=False, header=['probe_id'])

- based on `traceroute` measurements (see table `traceroutes` in `main.db`), identify probes that have private IP address as 1st hop and public IP address as 2nd hop
- save these probes to `../metadata/home-probes-identified.csv`
- merge both `../metadata/home-probes-ids.csv` (based on tag) and `../metadata/home-probes-identified.csv` (based on traces) into `../metadata/home-probes-merged.csv` (final list for analysis)

In [14]:
with sqlite3.connect('../data/main.db') as conn:
    df = pd.read_sql('select * from traceroute_measurements', con=conn)  # fetched traceroute results from Atlas API
home_probe_traces = df[df['ttl'].astype(int)<=2]

In [17]:
home_probe_traces

,msm_id,timestamp,method,probe_id,addr_fam,origin,src,dst,dst_name,status,ip_path_length,ttl,endpoint,rtt
0,22183901,1562029259,icmp-traceroute,10006,4,80.5.131.205,192.168.0.106,185.228.168.168,185.228.168.168,OK,13,1,192.168.0.1,46.039
1,22183901,1562029259,icmp-traceroute,10006,4,80.5.131.205,192.168.0.106,185.228.168.168,185.228.168.168,OK,13,2,*,
13,22183901,1562029250,icmp-traceroute,10019,4,176.141.164.243,192.168.31.41,185.228.168.168,185.228.168.168,OK,12,1,192.168.31.1,44.354
14,22183901,1562029250,icmp-traceroute,10019,4,176.141.164.243,192.168.31.41,185.228.168.168,185.228.168.168,OK,12,2,*,
25,22183901,1562029252,icmp-traceroute,10034,4,178.235.204.129,192.168.1.112,185.228.168.168,185.228.168.168,OK,13,1,192.168.1.1,50.024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6430922,22289109,1563191338,icmp-traceroute,33346,4,58.176.57.107,192.168.1.236,77.88.8.1,77.88.8.1,OK,7,2,58.176.56.1,1.577
6430928,22289109,1563192128,icmp-traceroute,34961,4,217.74.1.231,192.168.188.30,77.88.8.1,77.88.8.1,OK,6,1,*,
6430929,22289109,1563192128,icmp-traceroute,34961,4,217.74.1.231,192.168.188.30,77.88.8.1,77.88.8.1,OK,6,2,95.130.160.1,2.126
6430934,22289109,1563454936,icmp-traceroute,27827,4,94.179.11.17,192.168.1.2,77.88.8.1,77.88.8.1,OK,12,1,192.168.1.1,0.807


In [9]:
import ipaddress

In [27]:
def is_private(ipaddr):
    if ipaddr != '*':
        return ipaddress.ip_address(ipaddr).is_private
    return False

In [ ]:
home_probe_traces['address_private'] = home_probe_traces['endpoint'].map(is_private)

In [32]:
home_probe_traces

,msm_id,timestamp,method,probe_id,addr_fam,origin,src,dst,dst_name,status,ip_path_length,ttl,endpoint,rtt,address_private
0,22183901,1562029259,icmp-traceroute,10006,4,80.5.131.205,192.168.0.106,185.228.168.168,185.228.168.168,OK,13,1,192.168.0.1,46.039,True
1,22183901,1562029259,icmp-traceroute,10006,4,80.5.131.205,192.168.0.106,185.228.168.168,185.228.168.168,OK,13,2,*,,False
13,22183901,1562029250,icmp-traceroute,10019,4,176.141.164.243,192.168.31.41,185.228.168.168,185.228.168.168,OK,12,1,192.168.31.1,44.354,True
14,22183901,1562029250,icmp-traceroute,10019,4,176.141.164.243,192.168.31.41,185.228.168.168,185.228.168.168,OK,12,2,*,,False
25,22183901,1562029252,icmp-traceroute,10034,4,178.235.204.129,192.168.1.112,185.228.168.168,185.228.168.168,OK,13,1,192.168.1.1,50.024,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6430922,22289109,1563191338,icmp-traceroute,33346,4,58.176.57.107,192.168.1.236,77.88.8.1,77.88.8.1,OK,7,2,58.176.56.1,1.577,False
6430928,22289109,1563192128,icmp-traceroute,34961,4,217.74.1.231,192.168.188.30,77.88.8.1,77.88.8.1,OK,6,1,*,,False
6430929,22289109,1563192128,icmp-traceroute,34961,4,217.74.1.231,192.168.188.30,77.88.8.1,77.88.8.1,OK,6,2,95.130.160.1,2.126,False
6430934,22289109,1563454936,icmp-traceroute,27827,4,94.179.11.17,192.168.1.2,77.88.8.1,77.88.8.1,OK,12,1,192.168.1.1,0.807,True


In [36]:
tmp_1 = home_probe_traces[(home_probe_traces['ttl'].astype(int)==1) & (home_probe_traces['address_private'])]

In [37]:
tmp_1

,msm_id,timestamp,method,probe_id,addr_fam,origin,src,dst,dst_name,status,ip_path_length,ttl,endpoint,rtt,address_private
0,22183901,1562029259,icmp-traceroute,10006,4,80.5.131.205,192.168.0.106,185.228.168.168,185.228.168.168,OK,13,1,192.168.0.1,46.039,True
13,22183901,1562029250,icmp-traceroute,10019,4,176.141.164.243,192.168.31.41,185.228.168.168,185.228.168.168,OK,12,1,192.168.31.1,44.354,True
25,22183901,1562029252,icmp-traceroute,10034,4,178.235.204.129,192.168.1.112,185.228.168.168,185.228.168.168,OK,13,1,192.168.1.1,50.024,True
38,22183901,1562029226,icmp-traceroute,10101,4,213.126.51.234,10.0.1.25,185.228.168.168,185.228.168.168,OK,10,1,10.0.1.1,41.143,True
59,22183901,1562029239,icmp-traceroute,10131,4,87.250.37.34,192.168.39.253,185.228.168.168,185.228.168.168,OK,9,1,192.168.39.1,52.008,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6430897,22289109,1563190736,icmp-traceroute,19071,4,77.126.158.39,192.168.1.196,77.88.8.1,77.88.8.1,OK,7,1,192.168.1.1,6.819,True
6430904,22289109,1563190792,icmp-traceroute,34979,4,84.203.10.9,172.17.21.247,77.88.8.1,77.88.8.1,OK,5,1,172.17.21.254,0.786,True
6430909,22289109,1563191012,icmp-traceroute,11223,4,82.3.134.204,192.168.0.197,77.88.8.1,77.88.8.1,OK,12,1,192.168.0.1,0.588,True
6430921,22289109,1563191338,icmp-traceroute,33346,4,58.176.57.107,192.168.1.236,77.88.8.1,77.88.8.1,OK,7,1,192.168.1.1,0.577,True


In [38]:
tmp_2 = home_probe_traces[(home_probe_traces['ttl'].astype(int)==2) &
                          (home_probe_traces['endpoint'] != '*') &
                          (~home_probe_traces['address_private'])]

In [39]:
tmp_2

,msm_id,timestamp,method,probe_id,addr_fam,origin,src,dst,dst_name,status,ip_path_length,ttl,endpoint,rtt,address_private
39,22183901,1562029226,icmp-traceroute,10101,4,213.126.51.234,10.0.1.25,185.228.168.168,185.228.168.168,OK,10,2,213.126.51.233,8.807,False
49,22183901,1562029252,icmp-traceroute,10113,4,194.149.129.133,194.149.129.133,185.228.168.168,185.228.168.168,OK,11,2,194.149.130.2,0.657,False
60,22183901,1562029239,icmp-traceroute,10131,4,87.250.37.34,192.168.39.253,185.228.168.168,185.228.168.168,OK,9,2,87.250.37.45,4.496,False
69,22183901,1562029258,icmp-traceroute,10141,4,92.106.227.31,192.168.1.102,185.228.168.168,185.228.168.168,OK,12,2,62.203.135.1,8.215,False
81,22183901,1562029227,icmp-traceroute,10152,4,213.163.122.5,213.163.122.5,185.228.168.168,185.228.168.168,OK,9,2,213.163.96.25,2.372,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6430883,22289109,1563190284,icmp-traceroute,15938,4,145.236.114.55,192.168.1.244,77.88.8.1,77.88.8.1,OK,7,2,145.236.238.242,17.47,False
6430898,22289109,1563190736,icmp-traceroute,19071,4,77.126.158.39,192.168.1.196,77.88.8.1,77.88.8.1,OK,7,2,82.102.128.60,16.689,False
6430905,22289109,1563190792,icmp-traceroute,34979,4,84.203.10.9,172.17.21.247,77.88.8.1,77.88.8.1,OK,5,2,83.147.159.196,11.681,False
6430922,22289109,1563191338,icmp-traceroute,33346,4,58.176.57.107,192.168.1.236,77.88.8.1,77.88.8.1,OK,7,2,58.176.56.1,1.577,False


In [53]:
tmp_df = tmp_1.append(tmp_2)
tmp_df = tmp_df.sort_index().groupby(['msm_id', 'timestamp', 'probe_id', 'src', 'dst']).size().rename('entries').reset_index()

In [54]:
tmp_df

,msm_id,timestamp,probe_id,src,dst,entries
0,22183901,1562029206,24645,192.168.178.25,185.228.168.168,1
1,22183901,1562029206,28339,192.168.1.2,185.228.168.168,2
2,22183901,1562029207,33722,192.168.1.2,185.228.168.168,2
3,22183901,1562029208,22424,10.66.35.160,185.228.168.168,1
4,22183901,1562029208,35080,192.168.1.5,185.228.168.168,2
...,...,...,...,...,...,...
546141,22289109,1563190792,34979,172.17.21.247,77.88.8.1,2
546142,22289109,1563191012,11223,192.168.0.197,77.88.8.1,1
546143,22289109,1563191338,33346,192.168.1.236,77.88.8.1,2
546144,22289109,1563192128,34961,192.168.188.30,77.88.8.1,1


In [56]:
tmp_df[tmp_df['entries'] == 2]['probe_id'].unique()

array(['28339', '33722', '35080', ..., '28790', '21312', '25714'],
      dtype=object)

In [57]:
len(tmp_df[tmp_df['entries'] == 2]['probe_id'].unique())

2723

In [70]:
pd.DataFrame(tmp_df[tmp_df['entries'] == 2]['probe_id'].unique()
            ).rename(columns={0 : 'prb_id'}).sort_values('prb_id').to_csv('home-probes-identified.csv', index=False)

In [71]:
x = home_probes_ids.copy()

In [73]:
tmp_df[tmp_df['entries'] == 2]['probe_id']

1         28339
2         33722
4         35080
6         24093
11        25148
          ...  
546137    30957
546138    15938
546140    19071
546141    34979
546143    33346
Name: probe_id, Length: 217356, dtype: object

In [77]:
x.append(pd.DataFrame(tmp_df[tmp_df['entries'] == 2]['probe_id'].astype(int)))['probe_id'].unique()

array([10006, 10092, 10099, ..., 27270, 28790, 21312])

In [78]:
len(_)

3231

In [79]:
pd.DataFrame(x.append(pd.DataFrame(tmp_df[tmp_df['entries'] == 2]['probe_id'].astype(int)))['probe_id'].unique()
            ).rename(columns={0 : 'prb_id'}).sort_values('prb_id').to_csv('../metadata/home-probes-merged.csv',
                                                                          index=False)

In [82]:
probe_isps = df[['probe_id', 'origin']].drop_duplicates()

In [86]:
probe_isps

,probe_id,origin
0,10006,80.5.131.205
13,10019,176.141.164.243
25,10034,178.235.204.129
38,10101,213.126.51.234
48,10113,194.149.129.133
...,...,...
5692262,35598,122.199.58.220
5692359,35723,147.30.169.229
5692431,31289,203.132.89.57
5692489,30957,188.192.150.187


In [84]:
probe_isps_dedup = probe_isps.drop_duplicates('probe_id')

In [85]:
probe_isps_dedup

,probe_id,origin
0,10006,80.5.131.205
13,10019,176.141.164.243
25,10034,178.235.204.129
38,10101,213.126.51.234
48,10113,194.149.129.133
...,...,...
4058239,11526,109.132.147.74
4059657,12733,157.158.95.68
4892933,28790,93.205.174.31
5666711,27960,212.33.158.206


In [87]:
import pyasn

In [88]:
asndb = pyasn.pyasn('../metadata/ipasn-07-2019.dat')

In [89]:
def lookup_asn(ip):
    if ip is None:
        return None
    else:
        asn, prefix = asndb.lookup(ip)
        return asn

In [ ]:
probe_isps_dedup['asn'] = probe_isps_dedup['origin'].map(lookup_asn)

In [91]:
probe_isps_dedup

,probe_id,origin,asn
0,10006,80.5.131.205,5089
13,10019,176.141.164.243,5410
25,10034,178.235.204.129,29314
38,10101,213.126.51.234,33915
48,10113,194.149.129.133,5379
...,...,...,...
4058239,11526,109.132.147.74,5432
4059657,12733,157.158.95.68,8508
4892933,28790,93.205.174.31,3320
5666711,27960,212.33.158.206,8896


In [92]:
import json

In [93]:
with open('../metadata/as_names.json') as f:
    as_names_dict = json.load(f)
as_names_df = pd.DataFrame.from_dict(as_names_dict, orient='index').reset_index(level=0)
as_names_df.rename(columns={'index' : 'asn', 0: 'as_holder'}, inplace=True)
as_names_df['asn'] = as_names_df['asn'].astype(int)

In [94]:
probe_isps_dedup = probe_isps_dedup.merge(as_names_df, how='left', on='asn')

In [95]:
probe_isps_dedup

,probe_id,origin,asn,as_holder
0,10006,80.5.131.205,5089,"NTL, GB"
1,10019,176.141.164.243,5410,"BOUYGTEL-ISP, FR"
2,10034,178.235.204.129,29314,"VECTRANET-AS Al. Zwyciestwa 253, 81-525 Gdynia..."
3,10101,213.126.51.234,33915,"TNF-AS, NL"
4,10113,194.149.129.133,5379,"MK-UKIM-1, MK"
...,...,...,...,...
5166,11526,109.132.147.74,5432,"PROXIMUS-ISP-AS, BE"
5167,12733,157.158.95.68,8508,SILWEB-AS-EDU SILWEB Autonomous System - Acade...
5168,28790,93.205.174.31,3320,"DTAG Internet service provider operations, DE"
5169,27960,212.33.158.206,8896,"XFIBER-AS, NO"


In [98]:
probe_isps_dedup['as_holder'].value_counts()

COMCAST-7922, US                                                                223
LIBERTYGLOBAL Liberty Global (formerly UPC Broadband Holding, aka AORTA), AT    157
DTAG Internet service provider operations, DE                                   156
France Telecom - Orange, FR                                                     129
PROXAD, FR                                                                      113
                                                                               ... 
ASPETROVKA, UA                                                                    1
UMN-SYSTEM, US                                                                    1
SHARIF-AS, IR                                                                     1
LESSE, CA                                                                         1
BCS 32 rue Delizy, FR                                                             1
Name: as_holder, Length: 2083, dtype: int64

In [115]:
probe_isps_dedup['as_holder'].value_counts().head(20)

COMCAST-7922, US                                                                223
LIBERTYGLOBAL Liberty Global (formerly UPC Broadband Holding, aka AORTA), AT    157
DTAG Internet service provider operations, DE                                   156
France Telecom - Orange, FR                                                     129
PROXAD, FR                                                                      113
UUNET, US                                                                        60
KABELDEUTSCHLAND-AS, DE                                                          47
XS4ALL-NL Amsterdam, NL                                                          45
VERSATEL, DE                                                                     45
ATT-INTERNET4, US                                                                44
PROXIMUS-ISP-AS, BE                                                              41
NTL, GB                                                                     

In [102]:
home_probes = pd.read_csv('../metadata/home-probes-merged.csv')['prb_id'].unique()

In [103]:
home_probes

array([10006, 10007, 10080, ..., 35724, 35735, 35742])

In [109]:
home_probe_isps = probe_isps_dedup[probe_isps_dedup['probe_id'].astype(int).isin(home_probes)]

In [110]:
home_probe_isps

,probe_id,origin,asn,as_holder
0,10006,80.5.131.205,5089,"NTL, GB"
3,10101,213.126.51.234,33915,"TNF-AS, NL"
5,10131,87.250.37.34,35573,"AVCOM-AS, RS"
6,10141,92.106.227.31,3303,"SWISSCOM Swisscom (Switzerland) Ltd, CH"
9,10221,93.100.110.215,35807,"SKYNET-SPB-AS, RU"
...,...,...,...,...
5163,20622,185.237.164.6,35280,"ACORUS, FR"
5164,26475,59.166.114.142,9824,"JTCL-JP-AS Jupiter Telecommunication Co. Ltd, JP"
5165,13261,94.41.128.228,41704,"OGS-AS, RU"
5166,11526,109.132.147.74,5432,"PROXIMUS-ISP-AS, BE"


In [113]:
home_probe_isps['as_holder'].value_counts()

COMCAST-7922, US                                                                201
DTAG Internet service provider operations, DE                                   143
France Telecom - Orange, FR                                                     120
LIBERTYGLOBAL Liberty Global (formerly UPC Broadband Holding, aka AORTA), AT    112
PROXAD, FR                                                                      108
                                                                               ... 
BBIL-AP BHARTI Airtel Ltd., IN                                                    1
EVOK, CH                                                                          1
K-PSI-AS Conectio, PL                                                             1
IR-THR-PTE, IR                                                                    1
GAZETTE-RECORD-INTERNET, US                                                       1
Name: as_holder, Length: 1107, dtype: int64

In [114]:
home_probe_isps['as_holder'].value_counts().head(20)

COMCAST-7922, US                                                                201
DTAG Internet service provider operations, DE                                   143
France Telecom - Orange, FR                                                     120
LIBERTYGLOBAL Liberty Global (formerly UPC Broadband Holding, aka AORTA), AT    112
PROXAD, FR                                                                      108
UUNET, US                                                                        52
VERSATEL, DE                                                                     41
XS4ALL-NL Amsterdam, NL                                                          41
KABELDEUTSCHLAND-AS, DE                                                          40
PROXIMUS-ISP-AS, BE                                                              35
ATT-INTERNET4, US                                                                31
BOUYGTEL-ISP, FR                                                            

In [118]:
home_probe_isps.to_csv('../metadata/home-probe-isps.csv', index=False)

In [127]:
home_probe_isps.groupby(['asn', 'as_holder']
                       ).size().reset_index().rename(columns={0 : 'num'}
                                                    ).sort_values('num', ascending=False).head(20)

,asn,as_holder,num
165,7922,"COMCAST-7922, US",201
66,3320,"DTAG Internet service provider operations, DE",143
54,3215,"France Telecom - Orange, FR",120
146,6830,LIBERTYGLOBAL Liberty Global (formerly UPC Bro...,112
278,12322,"PROXAD, FR",108
8,701,"UUNET, US",52
61,3265,"XS4ALL-NL Amsterdam, NL",41
211,8881,"VERSATEL, DE",41
581,31334,"KABELDEUTSCHLAND-AS, DE",40
106,5432,"PROXIMUS-ISP-AS, BE",35


In [128]:
print(_.to_latex(index=False))

\begin{tabular}{rlr}
\toprule
   asn &                                          as\_holder &  num \\
\midrule
  7922 &                                   COMCAST-7922, US &  201 \\
  3320 &      DTAG Internet service provider operations, DE &  143 \\
  3215 &                        France Telecom - Orange, FR &  120 \\
  6830 &  LIBERTYGLOBAL Liberty Global (formerly UPC Bro... &  112 \\
 12322 &                                         PROXAD, FR &  108 \\
   701 &                                          UUNET, US &   52 \\
  3265 &                            XS4ALL-NL Amsterdam, NL &   41 \\
  8881 &                                       VERSATEL, DE &   41 \\
 31334 &                            KABELDEUTSCHLAND-AS, DE &   40 \\
  5432 &                                PROXIMUS-ISP-AS, BE &   35 \\
  7018 &                                  ATT-INTERNET4, US &   31 \\
  5410 &                                   BOUYGTEL-ISP, FR &   25 \\
  6848 &                                     TELEN